In [10]:
# -*- coding: utf-8 -*-

from skimage import io,transform
import glob
import os
import tensorflow as tf
import numpy as np
import time

#数据集地址
path='./Faces2/'
#模型保存地址
model_path='./Model/model2.ckpt'

#将所有的图片resize成100*100
w=100
h=100
c=3


#读取图片
def read_img(path):
    cate=[path+'/'+x for x in os.listdir(path) if os.path.isdir(path+'/'+x)]
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):
        for im in glob.glob(folder+'/*.jpg'):
            #print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
    
    print('img',img)
    print('labels',labels)
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)
data,label=read_img(path)


#打乱顺序
num_example=data.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
data=data[arr]
label=label[arr]


#将所有数据分为训练集和验证集
ratio=0.8
s=np.int(num_example*ratio)
x_train=data[:s]
y_train=label[:s]
x_val=data[s:]
y_val=label[s:]

#-----------------构建网络----------------------
#占位符
x=tf.placeholder(tf.float32,shape=[None,w,h,c],name='x')
y_=tf.placeholder(tf.int32,shape=[None,],name='y_')

#前向传播
def inference(input_tensor, train, regularizer):
    #卷积层
    #padding='SAME' 使用0来填充
    with tf.variable_scope('layer1-conv1',reuse=tf.AUTO_REUSE):
        conv1_weights = tf.get_variable("weight",[5,5,3,32],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases = tf.get_variable("bias", [32], initializer=tf.constant_initializer(0.0))
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))
    
    #池化层
    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="VALID")
    
    #卷积层
    with tf.variable_scope("layer3-conv2",reuse=tf.AUTO_REUSE):
        conv2_weights = tf.get_variable("weight",[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [64], initializer=tf.constant_initializer(0.0))
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    
    #池化层
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    #卷积层
    with tf.variable_scope("layer5-conv3",reuse=tf.AUTO_REUSE):
        conv3_weights = tf.get_variable("weight",[3,3,64,128],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv3_biases = tf.get_variable("bias", [128], initializer=tf.constant_initializer(0.0))
        conv3 = tf.nn.conv2d(pool2, conv3_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))
    
    #池化层
    with tf.name_scope("layer6-pool3"):
        pool3 = tf.nn.max_pool(relu3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #卷积层
    with tf.variable_scope("layer7-conv4",reuse=tf.AUTO_REUSE):
        conv4_weights = tf.get_variable("weight",[3,3,128,128],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv4_biases = tf.get_variable("bias", [128], initializer=tf.constant_initializer(0.0))
        conv4 = tf.nn.conv2d(pool3, conv4_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu4 = tf.nn.relu(tf.nn.bias_add(conv4, conv4_biases))

    #池化层
    with tf.name_scope("layer8-pool4"):
        pool4 = tf.nn.max_pool(relu4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        nodes = 6*6*128
        reshaped = tf.reshape(pool4,[-1,nodes])

    #全连接层
    with tf.variable_scope('layer9-fc1',reuse=tf.AUTO_REUSE):
        fc1_weights = tf.get_variable("weight", [nodes, 1024],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [1024], initializer=tf.constant_initializer(0.1))

        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train: fc1 = tf.nn.dropout(fc1, 0.5)

    #全连接层
    with tf.variable_scope('layer10-fc2',reuse=tf.AUTO_REUSE):
        fc2_weights = tf.get_variable("weight", [1024, 512],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [512], initializer=tf.constant_initializer(0.1))

        fc2 = tf.nn.relu(tf.matmul(fc1, fc2_weights) + fc2_biases)
        if train: fc2 = tf.nn.dropout(fc2, 0.5)
    
    #全连接层
    with tf.variable_scope('layer11-fc3',reuse=tf.AUTO_REUSE):
        fc3_weights = tf.get_variable("weight", [512, 115],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc3_weights))
        fc3_biases = tf.get_variable("bias", [115], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc2, fc3_weights) + fc3_biases

    return logit

#---------------------------网络结束---------------------------
regularizer = tf.contrib.layers.l2_regularizer(0.0001)
logits = inference(x,False,regularizer)

#(小处理)将logits乘以1赋值给logits_eval，定义name，方便在后续调用模型时通过tensor名字调用输出tensor
b = tf.constant(value = 1,dtype = tf.float32)
logits_eval = tf.multiply(logits,b,name='logits_eval') 

loss=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_)
train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1),tf.int32), y_)    
acc= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


#按批次取数据
def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]


#训练和测试数据，可将n_epoch（轮次）设置更大一些
n_epoch=10 
batch_size=64
saver=tf.train.Saver()
sess=tf.Session()  
sess.run(tf.global_variables_initializer())
for epoch in range(n_epoch):
    start_time = time.time()

    #training
    train_loss, train_acc, n_batch = 0, 0, 0
    for x_train_a, y_train_a in minibatches(x_train, y_train, batch_size, shuffle=True):
        _,err,ac=sess.run([train_op,loss,acc], feed_dict={x: x_train_a, y_: y_train_a})
        train_loss += err; train_acc += ac; n_batch += 1
    print("   train loss: %f" % (np.sum(train_loss)/ n_batch))
    print("   train acc: %f" % (np.sum(train_acc)/ n_batch))

    #validation
    val_loss, val_acc, n_batch = 0, 0, 0
    for x_val_a, y_val_a in minibatches(x_val, y_val, batch_size, shuffle=False):
        err, ac = sess.run([loss,acc], feed_dict={x: x_val_a, y_: y_val_a})
        val_loss += err; val_acc += ac; n_batch += 1
    print("   validation loss: %f" % (np.sum(val_loss)/ n_batch))
    print("   validation acc: %f" % (np.sum(val_acc)/ n_batch))
saver.save(sess,model_path)
sess.close()

reading the images:./Faces2//German_Khan/German_Khan_0001.jpg
reading the images:./Faces2//Stefano_Gabbana/Stefano_Gabbana_0001.jpg
reading the images:./Faces2//Dragan_Covic/Dragan_Covic_0001.jpg
reading the images:./Faces2//Jeff_Hornacek/Jeff_Hornacek_0001.jpg
reading the images:./Faces2//Sureyya_Ayhan/Sureyya_Ayhan_0001.jpg
reading the images:./Faces2//Deb_Santos/Deb_Santos_0001.jpg
reading the images:./Faces2//Bob_Newhart/Bob_Newhart_0001.jpg
reading the images:./Faces2//Wang_Hailan/Wang_Hailan_0001.jpg
reading the images:./Faces2//Paul_McNulty/Paul_McNulty_0002.jpg
reading the images:./Faces2//Paul_McNulty/Paul_McNulty_0001.jpg
reading the images:./Faces2//Jimmy_Iovine/Jimmy_Iovine_0001.jpg
reading the images:./Faces2//Claudia_Pechstein/Claudia_Pechstein_0001.jpg
reading the images:./Faces2//Claudia_Pechstein/Claudia_Pechstein_0003.jpg
reading the images:./Faces2//Claudia_Pechstein/Claudia_Pechstein_0002.jpg
reading the images:./Faces2//Claudia_Pechstein/Claudia_Pechstein_0005.jpg


/Users/pro/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


reading the images:./Faces2//Xiang_Huaicheng/Xiang_Huaicheng_0001.jpg
reading the images:./Faces2//Phil_Mickelson/Phil_Mickelson_0002.jpg
reading the images:./Faces2//Phil_Mickelson/Phil_Mickelson_0001.jpg
reading the images:./Faces2//Gerry_Kelly/Gerry_Kelly_0001.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0012.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0006.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0007.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0013.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0005.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0011.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0010.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0004.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0001.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0003.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hayek_0002.jpg
reading the images:./Faces2//Salma_Hayek/Salma_Hay

reading the images:./Faces2//Minnie_Mendoza/Minnie_Mendoza_0001.jpg
reading the images:./Faces2//Mark_Rosenbaum/Mark_Rosenbaum_0001.jpg
reading the images:./Faces2//Stephane_Rousseau/Stephane_Rousseau_0001.jpg
reading the images:./Faces2//Rita_Wilson/Rita_Wilson_0004.jpg
reading the images:./Faces2//Rita_Wilson/Rita_Wilson_0001.jpg
reading the images:./Faces2//Rita_Wilson/Rita_Wilson_0002.jpg
reading the images:./Faces2//Rita_Wilson/Rita_Wilson_0003.jpg
reading the images:./Faces2//Terence_Newman/Terence_Newman_0001.jpg
reading the images:./Faces2//Joseph_Biden/Joseph_Biden_0004.jpg
reading the images:./Faces2//Joseph_Biden/Joseph_Biden_0005.jpg
reading the images:./Faces2//Joseph_Biden/Joseph_Biden_0002.jpg
reading the images:./Faces2//Joseph_Biden/Joseph_Biden_0003.jpg
reading the images:./Faces2//Joseph_Biden/Joseph_Biden_0001.jpg
reading the images:./Faces2//Tommy_Tubberville/Tommy_Tubberville_0001.jpg
reading the images:./Faces2//Meles_Zenawi/Meles_Zenawi_0001.jpg
reading the imag

reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0011.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0007.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0013.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0012.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0006.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0002.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0016.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0017.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0003.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0015.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0001.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0014.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0019.jpg
reading the images:./Faces2//Jennifer_Aniston/Jennifer_Aniston_0

...

reading the images:./Faces2//Yu_Shyi-kun/Yu_Shyi-kun_0004.jpg
reading the images:./Faces2//Yu_Shyi-kun/Yu_Shyi-kun_0001.jpg
reading the images:./Faces2//Yu_Shyi-kun/Yu_Shyi-kun_0003.jpg
reading the images:./Faces2//Yu_Shyi-kun/Yu_Shyi-kun_0002.jpg
reading the images:./Faces2//Saeb_Erekat/Saeb_Erekat_0001.jpg
reading the images:./Faces2//Lou_Reed/Lou_Reed_0001.jpg
reading the images:./Faces2//Richard_Reid/Richard_Reid_0001.jpg
reading the images:./Faces2//Judi_Patton/Judi_Patton_0001.jpg
reading the images:./Faces2//Prince_William/Prince_William_0001.jpg
reading the images:./Faces2//Prince_William/Prince_William_0002.jpg
reading the images:./Faces2//James_Baker/James_Baker_0001.jpg
reading the images:./Faces2//John_Cusack/John_Cusack_0001.jpg
reading the images:./Faces2//John_Cusack/John_Cusack_0002.jpg
reading the images:./Faces2//Zarai_Toledo/Zarai_Toledo_0001.jpg
reading the images:./Faces2//Zarai_Toledo/Zarai_Toledo_0002.jpg
reading the images:./Faces2//Kelly_Leigh/Kelly_Leigh_0001.

reading the images:./Faces2//Gerhard_Schmid/Gerhard_Schmid_0001.jpg
reading the images:./Faces2//Raaf_Schefter/Raaf_Schefter_0001.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0041.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0040.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0036.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0022.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0023.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0037.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0009.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0021.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0035.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0034.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0020.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyton_Hewitt_0008.jpg
reading the images:./Faces2//Lleyton_Hewitt/Lleyto

reading the images:./Faces2//Ludwig_Ovalle/Ludwig_Ovalle_0001.jpg
reading the images:./Faces2//Michael_Caine/Michael_Caine_0003.jpg
reading the images:./Faces2//Michael_Caine/Michael_Caine_0002.jpg
reading the images:./Faces2//Michael_Caine/Michael_Caine_0001.jpg
reading the images:./Faces2//Michael_Caine/Michael_Caine_0004.jpg
reading the images:./Faces2//Agbani_Darego/Agbani_Darego_0001.jpg
reading the images:./Faces2//Abdullatif_Sener/Abdullatif_Sener_0002.jpg
reading the images:./Faces2//Abdullatif_Sener/Abdullatif_Sener_0001.jpg
reading the images:./Faces2//Zeljko_Rebraca/Zeljko_Rebraca_0001.jpg
reading the images:./Faces2//Tom_Gamboa/Tom_Gamboa_0001.jpg
reading the images:./Faces2//Marina_Silva/Marina_Silva_0001.jpg
reading the images:./Faces2//Tommy_Lewis/Tommy_Lewis_0001.jpg
reading the images:./Faces2//Bernard_Giraudeau/Bernard_Giraudeau_0001.jpg
reading the images:./Faces2//Giulio_Andreotti/Giulio_Andreotti_0001.jpg
reading the images:./Faces2//Jake_Plummer/Jake_Plummer_0001.

  [0.04746353 0.03427922 0.02373176]]]
labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 10, 10, 10, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 20, 21, 22, 22, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 26, 26, 26, 26, 26, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 34, 34, 34, 34, 34, 34, 35, 36, 36, 37, 38, 39, 40, 41, 42, 42, 43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 56, 57, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 67, 68, 69, 70, 71, 72, 73, 73, 74, 74, 74, 74, 75, 75, 76, 76, 77, 78, 78, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 81, 81, 82, 83, 84, 85, 86, 87, 88, 89, 89, 89, 89, 89, 89, 89, 90, 91, 92, 92, 92, 92, 92, 93, 94, 95, 96, 97, 97, 98, 99, 99, 99, 99, 99, 99, 99, 99, 99, 100, 101, 102, 102, 102, 103, 104, 105, 106, 106, 106, 106, 107, 108, 108, 108, 108, 108, 109, 110, 111, 112, 113, 113, 114, 115, 116, 116, 116, 116, 116, 116, 116, 117, 117, 117, 117, 117, 117, 117, 118, 119, 120, 1

ValueError: Variable layer1-conv1/weight/Adam/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-1-2f1e1270b27b>", line 146, in <module>
    train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
  File "/Users/pro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/pro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):
